From provided Azure subscription, connect to the resource group and list all of the resources 

In [ ]:
!pip install azure.identity azure.mgmt.resource

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

# Authenticate and create a client
credential = DefaultAzureCredential()
subscription_id = '829efd7e-aa80-4c0d-9c1c-7aa2557f8e07'
resource_group_name = 'github-copilot-web-apps'  # Replace with your resource group name
resource_client = ResourceManagementClient(credential, subscription_id)

resources = resource_client.resources.list_by_resource_group(resource_group_name)

r = []

for resource in resources:
    r.append(resource.id)
    print(f"Resource Name: {resource.name}, Resource Type: {resource.type}, Resource ID: {resource.id}")

Resource Name: privatelink.database.windows.net, Resource Type: Microsoft.Network/privateDnsZones, Resource ID: /subscriptions/1524a8ce-721b-46c1-ace6-5eda73aa74f9/resourceGroups/test/providers/Microsoft.Network/privateDnsZones/privatelink.database.windows.net
Resource Name: privatelink.redis.cache.windows.net, Resource Type: Microsoft.Network/privateDnsZones, Resource ID: /subscriptions/1524a8ce-721b-46c1-ace6-5eda73aa74f9/resourceGroups/test/providers/Microsoft.Network/privateDnsZones/privatelink.redis.cache.windows.net
Resource Name: testgosiaVnet, Resource Type: Microsoft.Network/virtualNetworks, Resource ID: /subscriptions/1524a8ce-721b-46c1-ace6-5eda73aa74f9/resourceGroups/test/providers/Microsoft.Network/virtualNetworks/testgosiaVnet
Resource Name: privatelink.database.windows.net/privatelink.database.windows.net-dblink, Resource Type: Microsoft.Network/privateDnsZones/virtualNetworkLinks, Resource ID: /subscriptions/1524a8ce-721b-46c1-ace6-5eda73aa74f9/resourceGroups/test/provi

for each resource id in r array, get the json from Azure and save it as a seperate file to the folder jsonFiles

In [18]:
import os
import json

# Ensure the directory exists
os.makedirs('jsonFiles', exist_ok=True)

for resource_id in r:
    resource = resource_client.resources.get_by_id(resource_id, api_version='2024-06-01')
    resource_json = resource.as_dict()
    
    # Create a file path based on the resource name
    file_name = f"jsonFiles/{resource.name}.json"
    
    # Write the JSON to the file
    with open(file_name, 'w') as f:
        json.dump(resource_json, f, indent=4)
    
    print(f"Saved {resource.name} to {file_name}")

Saved privatelink.database.windows.net to jsonFiles/privatelink.database.windows.net.json
Saved privatelink.redis.cache.windows.net to jsonFiles/privatelink.redis.cache.windows.net.json


HttpResponseError: (NoRegisteredProviderFound) No registered resource provider found for location 'uksouth' and API version '2024-06-01' for type 'virtualNetworks'. The supported api-versions are '2014-12-01-preview, 2015-05-01-preview, 2015-06-15, 2016-03-30, 2016-06-01, 2016-07-01, 2016-08-01, 2016-09-01, 2016-10-01, 2016-11-01, 2016-12-01, 2017-03-01, 2017-04-01, 2017-06-01, 2017-08-01, 2017-09-01, 2017-10-01, 2017-11-01, 2018-01-01, 2018-02-01, 2018-03-01, 2018-04-01, 2018-05-01, 2018-06-01, 2018-07-01, 2018-08-01, 2018-10-01, 2018-11-01, 2018-12-01, 2019-02-01, 2019-04-01, 2019-06-01, 2019-07-01, 2019-08-01, 2019-09-01, 2019-11-01, 2019-12-01, 2020-01-01, 2020-03-01, 2020-04-01, 2020-05-01, 2020-06-01, 2020-07-01, 2020-08-01, 2020-11-01, 2021-01-01, 2021-02-01, 2021-03-01, 2021-04-01, 2021-05-01, 2021-06-01, 2021-08-01, 2021-12-01, 2022-01-01, 2022-05-01, 2022-07-01, 2022-09-01, 2022-11-01, 2023-02-01, 2023-04-01, 2023-05-01, 2023-06-01, 2023-09-01, 2023-11-01, 2024-01-01, 2024-03

In [ ]:
# Install required packages
%pip install azure-identity azure-mgmt-resource

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:


from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

# Authenticate and create a client
credential = DefaultAzureCredential()
subscription_id = '829efd7e-aa80-4c0d-9c1c-7aa2557f8e07'
resource_group_name = 'github-copilot-web-apps-demo'  # Replace with your resource group name
resource_client = ResourceManagementClient(credential, subscription_id)

resources = resource_client.resources.list_by_resource_group(resource_group_name)

for resource in resources:
    print(f"Resource Name: {resource.name}, Resource Type: {resource.type}, Resource ID: {resource.id}")

ClientAuthenticationError: (InvalidAuthenticationTokenTenant) The access token is from the wrong issuer 'https://sts.windows.net/72f988bf-86f1-41af-91ab-2d7cd011db47/'. It must match one of the tenants 'https://sts.windows.net/2f4a9838-26b7-47ee-be60-ccc1fdec5953/,https://sts.windows.net/b4fd7cff-510b-4da5-b133-f7aa6f692ee2/' associated with this subscription. Please use any authority (URL) from 'https://login.windows.net/2f4a9838-26b7-47ee-be60-ccc1fdec5953,https://login.windows.net/b4fd7cff-510b-4da5-b133-f7aa6f692ee2' to get the token. Note, if the subscription is transferred to another tenant there is no impact to the services, but information about new tenant could take time to propagate (up to an hour). If you just transferred your subscription and see this error message, please try back later.
Code: InvalidAuthenticationTokenTenant
Message: The access token is from the wrong issuer 'https://sts.windows.net/72f988bf-86f1-41af-91ab-2d7cd011db47/'. It must match one of the tenants 

In [ ]:
!pip install mermaid-py

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
